In [1]:
from urllib import request, error, parse
from bs4 import BeautifulSoup
from datetime import date
import pandas as pd
import json
import csv
import re

In [ ]:
rn = date.today()
month = rn.strftime("%B").lower()
day = rn.day
year = rn.year

In [ ]:
current_date = "{}-{}-{}".format(month,day,year)
month_number = str(rn.month) if rn.month > 9 else '0' + str(rn.month)

In [ ]:
nj_dot_com_link = 'https://www.nj.com/coronavirus/2020/{0}/where-is-the-coronavirus-in-\
nj-latest-map-update-on-county-by-county-cases-{1}.html'.format(month_number, current_date)

In [ ]:

date = "{}-{}-{}".format(4,1,2020)
monthnumber = '04'

nj_dot_com_link = 'https://www.nj.com/coronavirus/2020/{0}/where-is-the-coronavirus-in-\
nj-latest-map-update-on-county-by-county-cases-{1}.html'.format(monthnumber, date)

In [ ]:
response = request.urlopen(nj_dot_com_link)
covid19_html = response.read()
soup = BeautifulSoup(covid19_html,'lxml')

In [ ]:
main = soup.find_all('main') # main part of article
article = soup.find_all('article') # article component

In [ ]:
a = [re.findall('.+\:\s*\d+',p.getText().lower()) for p in article[0].find_all('p') if ':' in str(p) and '•' in str(p)]
b = [p.getText().lower() for p in article[0].find_all('p') if ':' in str(p) and '•' in str(p)]

In [ ]:
data = []
for row in article[0].find_all('p'):
    delims = '.+\:\s*\d+'
    statistic = row.getText().lower()
    
    if ':' in statistic and '•' in statistic:
        cleaned_row = re.findall(delims,statistic)
        if len(cleaned_row) != 1:
            print(statistic)
            continue
        data.append(cleaned_row[0][1:].strip())

In [ ]:
nj_municipals = set(line.strip() for line in open('nj_municipals.txt'))

In [ ]:
substring_errors = {
    'pemberton boro' : 'pemberton',
    ' borough' : '',
    'parsippany' : 'parsippany-troy hills',
    ' city' : '',
    'south orange' : 'south orange village',
    'oldsman' : 'oldmans',
    'peapack-gladstone' : 'peapack and gladstone',
    'bryram' : 'byram',
    'wantgage' : 'wantage'
}

fullstring_errors = {
    'orange' : 'city of orange',
    'clinton town' : 'clinton township'
}

In [ ]:
nj_covid_19_data = {}

for row in data:
    infected_township = re.split(':',row)
    town = infected_township[0].strip()
    num_infected = int(infected_township[1].strip())
    
    for error in substring_errors.keys():
        town = town.replace(error,errors[error])
        
    for error in fullstring_errors.keys():
        if town.strip() == error:
            town = fullstring_errors[error]
        
    town = town.strip()
    township = town + ' township'
    city = town + ' city'
    
    if town in nj_municipals:
        nj_covid_19_data[town] = num_infected
        
    elif township in nj_municipals:
        nj_covid_19_data[township] = num_infected
        
    elif city in nj_municipals:
        nj_covid_19_data[city] = num_infected
    else: 
        print('ERROR TOWN NOT FOUND IN NJ MUNICIPALS: {}'.format(town))

In [ ]:
json_data = {
    'last fetched' : current_date,
    'data' : nj_covid_19_data
}

In [ ]:
with open('covid19_nj.json', 'w') as datafile:
    json.dump(json_data,datafile)